# Inference on model and store the output for further use
Use for deep ensemble method, run model on test data set and store the output into json file
The original DINO code did not support save_results function

In [11]:
import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops
import pickle

In [24]:
scale_config = "5scale"
checkpoint = "0031"
model_config_path = "config/DINO/DINO_" + scale_config + ".py" # change the path of the model config file
model_checkpoint_path = "ckpts/checkpoint" + checkpoint + "_" + scale_config + ".pth" # change the path of the model checkpoint
results_path = "./results/deep_ensemble/" + "DINO_" + checkpoint + "_" + scale_config + ".pkl"
thershold = 0.3 # set a thershold
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 

## Load data set

In [25]:
args.dataset_file = 'coco'
args.coco_path = "../coco/" # the path of coco
args.fix_size = False

dataset_val = build_dataset(image_set='val', args=args)

data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!


## Load model

In [26]:
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

In [27]:
output_dic = {'pred_logits', 'pred_boxes'}
results = {}
with torch.no_grad():
    for idx in range(len(dataset_val)):
        test_result = {}
        output = model.cuda()(dataset_val[idx][0][None].cuda())
        for k in output_dic:
            test_result[k] = output[k].cpu().numpy()
        output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]
        boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
        test_result['scores'] = output['scores'].cpu().numpy()
        test_result['labels'] = output['labels'].cpu().numpy()
        test_result['boxes'] = boxes.cpu().numpy()
        results[idx] = test_result
        if (idx+1) % 1000 == 0:
            print(f"Complete test data {idx}")

with open(results_path, "wb") as outfile:
     pickle.dump(results, outfile)
print(f"Complete save results into {results_path}")

Complete test data 999
Complete test data 1999
Complete test data 2999
Complete test data 3999
Complete test data 4999
Complete save results into ./results/deep_ensemble/DINO_0031_5scale.pkl


In [36]:
import pickle
results_path = "./results/deep_ensemble/" + "DINO_" + checkpoint + "_" + scale_config + ".pkl"
with open(results_path, "wb") as outfile:
     pickle.dump(results, outfile)

## SWIN

In [30]:
scale_config = "4scale_swin"
checkpoint = "0029"
model_config_path = "config/DINO/DINO_" + scale_config + ".py" # change the path of the model config file
model_checkpoint_path = "ckpts/checkpoint" + checkpoint + "_" + scale_config + ".pth" # change the path of the model checkpoint
results_path = "./results/deep_ensemble/" + "DINO_" + checkpoint + "_" + scale_config + ".pkl"
thershold = 0.3 # set a thershold
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
args.backbone_dir = "./"

# load data
args.dataset_file = 'coco'
args.coco_path = "../coco/" # the path of coco
args.fix_size = False

dataset_val = build_dataset(image_set='val', args=args)

# load model
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.44s)
creating index...
index created!
use_checkpoint!!!!!!!!!!!!!!!!!!!!!!!!
_IncompatibleKeys(missing_keys=['norm1.weight', 'norm1.bias', 'norm2.weight', 'norm2.bias', 'norm3.weight', 'norm3.bias'], unexpected_keys=['norm.weight', 'norm.bias', 'layers.0.blocks.1.attn_mask', 'layers.1.blocks.1.attn_mask', 'layers.2.blocks.1.attn_mask', 'layers.2.blocks.3.attn_mask', 'layers.2.blocks.5.attn_mask', 'layers.2.blocks.7.attn_mask', 'layers.2.blocks.9.attn_mask', 'layers.2.blocks.11.attn_mask', 'layers.2.blocks.13.attn_mask', 'layers.2.blocks.15.attn_mask', 'layers.2.blocks.17.attn_mask'])


In [31]:
output_dic = {'pred_logits', 'pred_boxes'}
results = {}
with torch.no_grad():
    for idx in range(len(dataset_val)):
        test_result = {}
        output = model.cuda()(dataset_val[idx][0][None].cuda())
        for k in output_dic:
            test_result[k] = output[k].cpu().numpy()
        output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]
        boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
        test_result['scores'] = output['scores'].cpu().numpy()
        test_result['labels'] = output['labels'].cpu().numpy()
        test_result['boxes'] = boxes.cpu().numpy()
        results[idx] = test_result
        if (idx+1) % 1000 == 0:
            print(f"Complete test data {idx}")

with open(results_path, "wb") as outfile:
     pickle.dump(results, outfile)
print(f"Complete save results into {results_path}")

Complete test data 999
Complete test data 1999
Complete test data 2999
Complete test data 3999
Complete test data 4999
Complete save results into ./results/deep_ensemble/DINO_0029_4scale_swin.pkl
